In [ ]:
from typing import List, Tuple
from datetime import datetime, timedelta
import os
import cv2
import pickle
import itertools
import json
import multiprocessing
from math import nan

import altair as alt
import numpy.typing as npt
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils.compose2_factory import compose2_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos
from utils.frame_json_encoder import FrameJSONEncoder
from utils.typings import Frame
from utils.load_images import load_images

In [ ]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

In [ ]:
fps = 20

In [ ]:
interpolate = False

In [ ]:
partition_dir = os.path.join(processed_dir, 'partition')

In [ ]:
compose = compose2_factory(fps, raw_dir, processed_dir, interpolate)

In [ ]:
def iframe(timestamp: "datetime"):
    return Frame(
        scene=None,
        token=None,
        frame_order=nan,
        filename=None,
        camera_translation=(nan, nan, nan),
        camera_rotation=(nan, nan, nan, nan),
        camera_intrinsic=((nan, nan, nan), (nan, nan, nan), (nan, nan, nan)),
        ego_translation=(nan, nan, nan),
        ego_rotation=(nan, nan, nan, nan),
        timestamp=timestamp,
        camera_heading=nan,
        ego_heading=nan
    )

In [ ]:
all_misalignments = []
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    filenames = []
    for name, video in videos.items():
#         if 'scene-1077-CAM_BACK' not in name:
#             continue
        for f in video:
            filenames.append(f.filename)
    
    img_cache = load_images(raw_dir, filenames)
    print("Done Loading Images")
    
#     with multiprocessing.Pool() as pool:
#         inputs = ((l, name, video, img_cache, compose) for name, video in videos.items())
#         _outputs = tqdm(pool.imap_unordered(process_one_video, inputs), total=len(videos))
#         output = {
#             k: v
#             for k, v
#             in _outputs
#         }
    
    output = {}
    misalignments = []
    for name in videos:
#         if 'scene-1077-CAM_BACK' not in name:
#             continue
        video = videos[name]
        frames, filename, _misalignments = compose(l, name, video, img_cache)
        misalignments.extend(_misalignments)
#         prev = video[0][9]
#         for v in video[1:]:
#             print(int(round((v[9] - prev).total_seconds() * 100,)))
#             prev = v[9]
        o = {
            "filename": filename,
            "start": video[0][9],
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": [
                (tuple(iframe(f)) if isinstance(f, datetime) else tuple(video[f]))
                for img, f in frames
            ]
        }
#         print('---------------------------')
        prev = o['frames'][0][9]
        for i, v in enumerate(o['frames'][1:30]):
#             print(int(round((v[9] - prev).total_seconds() * 100,)))
            assert int(round((v[9] - prev).total_seconds() * 100)) == 5, (int(round((v[9] - prev).total_seconds() * 100,)), i + 1, v)
            prev = v[9]
        output[name] = o
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames-skip.pickle'), "wb") as f:
        pickle.dump(output, f)
    
    with open(os.path.join(base, 'frames-skip.json'), "w") as f:
        json.dump(output, f, indent=2, sort_keys=True, cls=FrameJSONEncoder)
    
    # with open(os.path.join(base, 'misalignments.json'), "w") as f:
    #     json.dump([{'misalignment': m} for m in misalignments], f, indent=2)
    all_misalignments.extend([{'misalignment': m, "location": l} for m in misalignments])

In [ ]:
# TODO: Partition data based on location
alt.Chart(pd.DataFrame(all_misalignments)).mark_tick().encode(
    alt.X('misalignment:Q', title='Misalignment (%)'),
    alt.Y('location:Q', title='Location'),
)